In [ ]:
import yfinance as yf

In [ ]:
sp500=yf.Ticker("^GSPC")

In [ ]:
sp500=sp500.history(period="max")

In [ ]:
sp500

In [ ]:
sp500.index

In [ ]:
sp500.plot.line(y="Close",use_index=True)

In [ ]:
del sp500['Dividends']
del sp500['Stock Splits']

In [ ]:
sp500["Tomorrow"]=sp500["Close"].shift(-1)

In [ ]:
sp500

In [ ]:
sp500['target']=(sp500['Tomorrow']> sp500["Close"]).astype(int)
sp500

In [ ]:
sp500=sp500.loc["1990-01-01":].copy()
sp500

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier(n_estimators=100, min_samples_split=100,random_state=1)

train = sp500.iloc[:-100]
test = sp500.iloc[-100:]


predictors=['Close','Volume',"Open",'High',"Low"]
model.fit(train[predictors],train["target"])

In [ ]:
from sklearn.metrics import precision_score
preds=model.predict(test[predictors])

In [ ]:
import pandas as pd
preds=pd.Series(preds, index=test.index)


In [ ]:
precision_score(test["target"],preds)

In [ ]:
combined=pd.concat([test["target"],preds],axis=1)

In [ ]:
combined.plot()

**BackTesting**

In [ ]:
def predict(train,test,predictors,model):
  model.fit(train[predictors],train["target"])
  preds=model.predict(test[predictors])
  preds=pd.Series(preds,index=test.index,name="predictions")
  combined=pd.concat([test["target"],preds],axis=1)
  return combined

In [ ]:
def backtest(data,model,predictors,start=2500,step=250):
  all_predictions=[]
  for i in range(start,data.shape[0],step):
    train=data.iloc[0:i].copy()
    test=data.iloc[i:(i+step)].copy()
    predictions=predict(train,test,predictors,model)
    all_predictions.append(predictions)
  return pd.concat(all_predictions)

In [ ]:
predictions = backtest(sp500,model,predictors)

In [ ]:
predictions['predictions'].value_counts()

In [ ]:
precision_score(predictions["target"],predictions['predictions'])